In [9]:
import csv

In [10]:
# Functions to extract bits and compute the two's compliment

def twos_comp(val, bits):
    """compute the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val                         # return positive value as is

def extract_value(val, startBit, numBits):
    """extract a specified number of bits from an int, starting at startBit (counting from 0 on the right)"""
    bits = int(bin((2**numBits)-1),2)
    val = bin(val >> startBit & bits)
    return int(val,2)

In [15]:
# Set eye scan parameters
dataWidth = 32

base_path = "finalcode/gearshift/EoS/testing/"
# Set the input file name with scan data
filename = "prescale9defaultConfig.csv"
inputFile = base_path + filename
# Set output filename
path_to_plots = "finalcode/gearshift/EoS/testing/plots/"
outputFilename = path_to_plots+ "eyeData" + filename


In [16]:
# Open csv of scan data and extract results
# Data to be extracted:
# - Samples 16bit hex
# - Errors 16bit hex
# - Vertical offset 7bit hex
# - Vertical offset sign 1bit (negative or positive)
# - Horizontal offset 11bit hex 2's compliment
# - Prescale 5bit hex
# - UTsign 1bit

samples0 = []
samples1 = []
negative = 1

with open(inputFile, newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    next(spamreader, None)
    next(spamreader, None)
    for row in spamreader:
        if "00" not in row[0].split(',')[3]:
            sample = int(row[0].split(',')[5], 16)
            error = int(row[0].split(',')[6], 16)
            hHex = int(row[0].split(',')[7],16)
            hExtract = extract_value(hHex, 0, 11)
            hcomp = twos_comp(hExtract, 11)
            vOffset = int(row[0].split(',')[8], 16)
            prescale = extract_value(vOffset, 11, 5)
            UTsign = extract_value(vOffset, 8, 1)
            offsetSign = extract_value(vOffset, 7, 1)
            vOffset = extract_value(vOffset, 0, 7)
            if offsetSign:
                negative = -1
            else:
                negative = 1
            if (UTsign==0):
                samples0.append((sample, error, hcomp, ((vOffset)*negative), prescale))
            else:
                samples1.append((sample, error, hcomp, ((vOffset)*negative), prescale))

In [17]:
# Set up the vertical and horizontal indices
verticals = []
horizontals = []
for tup in samples1:
    verticals.append(int(tup[3]))
    horizontals.append(int(tup[2]))
# Remove duplicates
verticals = (list(dict.fromkeys(verticals)))
horizontals = (list(dict.fromkeys(horizontals)))
# Order horizontals from lowest to highest
horizontals.sort(reverse=True)


In [18]:
# Calculate the BER and write the results to the output file
with open(outputFilename, 'w+', newline="") as outcsv:
    filewriter = csv.writer(outcsv, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(["Offsets",-32,-31,-30,-29,-28,-27,-26,-25,-24,-23,-22,-21,-20,-19,-18,-17,-16,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32])
    for vert in verticals:
        sample0 = 0
        sample1 = 0
        error0 = 0
        error1 = 0
        berList = []
        berList.append(vert)
        for horz in horizontals:
            # Get total samples by multiplying sample count with data width and 2^(prescale + 1)
            for sample in samples0:
                if sample[2] == horz and sample[3] == vert:
                    sample0 = sample[0]
                    error0 = sample[1]
                    prescale0 = sample[4]
            totalSamples0 = sample0 * dataWidth * 2**(prescale0 + 1)
            
            if totalSamples0 == 0 :
                berList.append("error 0")
                continue

            for sample in samples1:
                if sample[2] == horz and sample[3] == vert:
                    sample1 = sample[0]
                    error1 = sample[1]
                    prescale1 = sample[4]
            totalSamples1 = sample1 * dataWidth * 2**(prescale1 + 1)
            
            if totalSamples1 == 0 :
                berList.append("error 1")
                continue

            # Calculate BER for ut-sign of 0
            ber0 = error0/totalSamples0
            
            # Calculate BER for ut-sign of 1
            ber1 = error1/totalSamples1

            ber = (ber0 + ber1)/2

            # Handle case where errors = 0
            if ber == 0:
                ber = 1/(totalSamples0 + totalSamples1)
            berList.append(ber)
        filewriter.writerow(berList)